In [1]:
import warnings
import joblib
from compare import *
import pandas as pd
import numpy as np
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
from sklearn.svm import LinearSVR,SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor,VotingRegressor, IsolationForest
from sklearn.feature_selection import f_regression
from sklearn.metrics import mean_squared_error,mean_absolute_error, mean_absolute_percentage_error, r2_score
sns.set_style('whitegrid')
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
#load your data into the dataframe
my_data = read_csv('COM726.csv')

In [3]:
# Drop column with Nan value
my_data = my_data.drop(['Unnamed: 16'], axis=1)
my_data.head()

,Region_Country,Second_Tier_Authority,Local_Authority,Code,Year,Industry_Total,Commercial_Total,Public_Sector_Total,Domestic_Total,Transport_Total,LULUCF_Net_Emissions,Grand_Total,Population,Per_Capita_Emissions(t),Area(km2),Emissions_per_km2(kt)
0,North East,Darlington,Darlington,E06000005,2005.0,188.7,104.9,47.5,262.5,214.0,-0.9,816.7,100.3,8.1,197.5,4.1
1,North East,Darlington,Darlington,E06000005,2006.0,175.9,108.2,46.7,260.3,210.1,-1.4,799.8,101.5,7.9,197.5,4.1
2,North East,Darlington,Darlington,E06000005,2007.0,170.7,102.8,44.7,251.2,208.4,-2.0,775.7,102.6,7.6,197.5,3.9
3,North East,Darlington,Darlington,E06000005,2008.0,177.3,101.8,44.3,252.2,193.1,-2.3,766.4,103.7,7.4,197.5,3.9
4,North East,Darlington,Darlington,E06000005,2009.0,149.2,89.3,38.7,228.8,190.2,-2.3,693.9,104.4,6.6,197.5,3.5


In [4]:
# Drop rows which have all NaN in its row
my_data = my_data.dropna(how='all')

In [5]:
# Drop columns
my_data = my_data.drop(['Region_Country', 'Second_Tier_Authority', 'Local_Authority', 'Code', 'Grand_Total'], axis=1)
my_data.head()

,Year,Industry_Total,Commercial_Total,Public_Sector_Total,Domestic_Total,Transport_Total,LULUCF_Net_Emissions,Population,Per_Capita_Emissions(t),Area(km2),Emissions_per_km2(kt)
0,2005.0,188.7,104.9,47.5,262.5,214.0,-0.9,100.3,8.1,197.5,4.1
1,2006.0,175.9,108.2,46.7,260.3,210.1,-1.4,101.5,7.9,197.5,4.1
2,2007.0,170.7,102.8,44.7,251.2,208.4,-2.0,102.6,7.6,197.5,3.9
3,2008.0,177.3,101.8,44.3,252.2,193.1,-2.3,103.7,7.4,197.5,3.9
4,2009.0,149.2,89.3,38.7,228.8,190.2,-2.3,104.4,6.6,197.5,3.5


In [6]:
my_data['Year'] = my_data['Year'].apply(np.int64)

print(my_data.dtypes)

Year                         int64
Industry_Total             float64
Commercial_Total           float64
Public_Sector_Total        float64
Domestic_Total             float64
Transport_Total            float64
LULUCF_Net_Emissions       float64
Population                 float64
Per_Capita_Emissions(t)    float64
Area(km2)                  float64
Emissions_per_km2(kt)      float64
dtype: object


In [7]:
X = my_data.drop('Per_Capita_Emissions(t)', axis=1)  #  X= my_data.iloc[:,:-1]
y = my_data['Per_Capita_Emissions(t)']  # y= my_data.iloc[:,-1]

In [8]:
#split the data into train and test - split using 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
#check the number of train and test data
print('\n The total of training dataset', X_train.shape)
print('\n The total of test dataset', X_test.shape)


 The total of training dataset (3979, 10)

 The total of test dataset (1706, 10)


In [10]:
iforest = IsolationForest(random_state=0)
y_pred = iforest.fit_predict(X_train)

In [13]:
X_train_iforest, y_train_iforest = X_train.iloc[(y_pred != -1), :], y_train.iloc[(y_pred != -1)]

In [14]:
print(X_train_iforest.shape, y_train_iforest.shape)

(3659, 10) (3659,)


In [16]:
rf_model = RandomForestRegressor(random_state=0)
#fit the model to the training set
model1 = rf_model.fit(X_train, y_train)
#predict the model
y_pred = model1.predict(X_test)
#predict model on train data
y_pred_train = model1.predict(X_train)
#evaluate
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
#the r2 for the train dataset
r2_train = r2_score(y_train, y_pred_train)

#printing the model evaluation values
print('Mean absolute error: {:.2f}'.format(mae))
print('Mean squared error: {:.2f}'.format(mse))
print('Root mean squared error: {:.2f}'.format(rmse))
print('Mean absolute percentage error:, {:.2f}'.format(mape))
print('r2 score:{:.2f}'.format(r2))
print('R2 Score Train data:{:.2f}'.format(r2_train))

Mean absolute error: 0.35
Mean squared error: 2.51
Root mean squared error: 1.58
Mean absolute percentage error:, 0.04
r2 score:0.97
R2 Score Train data:1.00
